connection a la base de données 

In [ ]:
import mysql.connector

# Connexion à la base de données MySQL
mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="datascientest",
  password="temp123",
  database="opa"
)

# Créer un curseur pour exécuter les requêtes SQL
mycursor = mydb.cursor()

# Exécuter une requête SQL pour récupérer les données
mycursor.execute("SELECT * FROM opa.historical_klines;")

# Récupérer les résultats
resultats = mycursor.fetchall()

# Fermer la connexion à la base de données
mydb.close()
# Transformer les résultats en DataFrame pandas
df = pd.DataFrame(resultats, columns=["id_symint", "open_time", "open_price", "high_price", "low_price", 
                                      "close_price", "volume", "close_time", "quote_asset_volume", 
                                      "number_of_trades", "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume"])

# Exporter le DataFrame dans un fichier CSV
df.to_csv("donnes binance historiques.csv", index=False)

